In [ ]:
import pandas as pd
import numpy as np
transaction=pd.read_csv('../input/train_2016.csv', parse_dates=["transactiondate"]);
transaction['sale_month']=transaction['transactiondate'].apply(lambda x: (x.to_datetime()).month)
transaction['sale_day']=transaction['transactiondate'].apply(lambda x: (x.to_datetime()).day)
transaction['sale_year']=transaction['transactiondate'].apply(lambda x: (x.to_datetime()).year)
transaction['sale_month'].astype(int,inplace=True)
transaction.drop(['transactiondate','sale_day','sale_year'],axis=1,inplace=True)
properties_2016=pd.read_csv('../input/properties_2016.csv',low_memory=False);
# data type analysis
dtype_df = properties_2016.dtypes.reset_index()
dtype_df.columns = ["Feature", "Column Type"]
dtype_df.groupby("Column Type").aggregate('count').reset_index()
properties_2016.drop(dtype_df[dtype_df['Column Type']=='object']['Feature'].values.tolist(),axis=1,inplace=True)
# check the missing percentage
missing_df = properties_2016.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df = missing_df.ix[missing_df['missing_count']>0]
missing_df = missing_df.sort_values(by='missing_count')
missing_df['missing_rate']=missing_df['missing_count']/2985217
cutoff=0.9
# drop feature missing rate>0.9
properties_2016.drop(missing_df[(missing_df.missing_rate>=cutoff)].column_name.values.tolist(),
                    axis=1,inplace=True)
# categorize the left feature
id_feature_left=['propertylandusetypeid','assessmentyear','yearbuilt',
                'buildingqualitytypeid','heatingorsystemtypeid','fips','rawcensustractandblock',
                'regionidzip','regionidcity']
cnt_feature_left=['bedroomcnt','bathroomcnt','roomcnt','calculatedbathnbr','fullbathcnt',
                 'unitcnt','garagecarcnt','airconditioningtypeid','numberofstories',
                  'poolcnt','pooltypeid7','fireplacecnt','threequarterbathnbr']
size_feature_left=['taxamount','taxvaluedollarcnt','structuretaxvaluedollarcnt',
                   'calculatedfinishedsquarefeet','landtaxvaluedollarcnt','finishedsquarefeet12',
                  'lotsizesquarefeet','longitude','latitude','garagetotalsqft']
location_feature_left=['regionidcounty','censustractandblock','regionidneighborhood']
# fill missing values
# for id_feature, fill the missing values with most frequent value
# for cnt_feature, fill the missing values with median value
# for size_feature, fill the missing values with mean values
# for location_feature, fill the missing values with the nearest values
fill_missing_value=dict()
# for id_feature
for x in id_feature_left:
    fill_missing_value[x]=properties_2016[x].value_counts().index.tolist()[0]
# for cnt_feature
for x in cnt_feature_left:
    fill_missing_value[x]=properties_2016[x].median()
# for size_feature
for x in size_feature_left:
    fill_missing_value[x]=properties_2016[x].mean()
for x in fill_missing_value:
    properties_2016[x].fillna(fill_missing_value[x],inplace=True)
# for location_feature
# regionidcounty and fips is the same to represent the county keep fips
# censustractandblock is drop as to be the same as rawcensustractandblock.
# regionidneighborhood >60% missing rate is droped
properties_2016.drop(['regionidcounty','regionidneighborhood','censustractandblock'],axis=1,inplace=True)
# the maximum missing rate for the left location feature is 2% maximum. drop the rows with
# missing values
#properties_2016.dropna(axis=0,how='any',inplace=True)
#properties_2016.fillna(0,inplace=True)
# divide 1000000 for longitude and latitude
properties_2016['longitude']=properties_2016['longitude']/1000000;
properties_2016['latitude']=properties_2016['latitude']/1000000;
# add new features
# 1. tax per living area = tax amount/calculatedfinishedsquarefeet
# 2. tax per living area2 =tax amount/finishedsquarefeet12
# 3. tax per lot size=taxamount/lotsizesquarefeet
properties_2016['tax_per_liv_area']=properties_2016['taxamount']/properties_2016['calculatedfinishedsquarefeet'];
properties_2016['tax_per_liv_area2']=properties_2016['taxamount']/properties_2016['finishedsquarefeet12'];
properties_2016['tax_per_lot_size']=properties_2016['taxamount']/properties_2016['lotsizesquarefeet'];
# achieve data with transaction information
train_df=transaction.merge(properties_2016,on='parcelid',how='left')
# drop missing value which indicates these properties have no complete location information
train_df.dropna(axis=0,how='any',inplace=True)
print((len(transaction)-len(train_df))/len(transaction)*100,'% data has been removed due to the missing',
     'location information.')
# drop transactiondata, sale_year
#train_df.drop(['transactiondate','sale_year'],axis=1,inplace=True)
# drop sale_day since there is not enough data to resolve resolution in day
#train_df.drop('sale_day',axis=1,inplace=True)
# set parcel id as index
train_df.set_index('parcelid',inplace=True)
# drop pooltypeid7
properties_2016.drop('pooltypeid7',inplace=True,axis=1)
properties_2016.drop('poolcnt',inplace=True,axis=1)
#  7+12+3+14+15
dummy_feature=['airconditioningtypeid','buildingqualitytypeid','fips','heatingorsystemtypeid',
              'propertylandusetypeid']

# normalize all features except dummy feature
features=properties_2016.columns.tolist()
for x in list(set(features)-set(dummy_feature)-set(['pooltypeid7','poolcnt','parcelid'])):
    feature_max=properties_2016[x].max()
    feature_min=properties_2016[x].min()
    if(feature_max==feature_min):
        print(x,'max==min, cannot normalize, max=',feature_max)
    else:
        properties_2016[x]=(properties_2016[x]-feature_min)/(feature_max-feature_min)
# add dummy variables
for x in dummy_feature:
    a=pd.get_dummies(properties_2016[x])
    print(a.shape)
    a.columns=[x+'_'+str(n) for n in a.columns.tolist()]
    properties_2016=pd.concat([properties_2016,a], axis=1)
print("new properties shape:",properties_2016.shape)
# drop dummy feature
properties_2016.drop(dummy_feature,axis=1,inplace=True)
train_df=transaction.merge(properties_2016,on='parcelid',how='left')
#train_df.drop(['transactiondate','sale_year'],axis=1,inplace=True)
# drop sale_day since there is not enough data to resolve resolution in day
#train_df.drop('sale_day',axis=1,inplace=True)
# set parcel id as index
train_df.set_index('parcelid',inplace=True)
train_df['sale_month']=train_df['sale_month']/12;
# add one column as the absolute value of log error
#train_df['abs_logerror']=train_df['logerror'].abs()
# get X_train y_train
X_train=train_df.iloc[:,1:]
#X_train.set_index('parcelid',inplace=True)
y_train=train_df['logerror']
# generate X_test
# load sample submission for output
sample = pd.read_csv('../input/sample_submission.csv')
sample['parcelid'] = sample['ParcelId']
df_test = sample.merge(properties_2016, on='parcelid', how='left')
# still need the sale month as input
train_feature=properties_2016.columns.tolist()
X_test=df_test[train_feature]
X_test.set_index('parcelid',inplace=True)
# predict with linear regress
X_test['sale_month']=10/12
X_test=X_test[X_train.columns.tolist()]

In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import gc
split = 90000
x_train, y_train, x_valid, y_valid = X_train[:split], y_train[:split], X_train[split:], y_train[split:]
x_train = x_train.values.astype(np.float32, copy=False)
x_valid = x_valid.values.astype(np.float32, copy=False)

d_train = lgb.Dataset(x_train, label=y_train)
d_valid = lgb.Dataset(x_valid, label=y_valid)

params = {}
params['learning_rate'] = 0.002
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'mae'
params['sub_feature'] = 0.5
params['num_leaves'] = 60
params['min_data'] = 500
params['min_hessian'] = 1

watchlist = [d_valid]
clf = lgb.train(params, d_train, 500, watchlist)

In [ ]:
output=pd.read_csv('../input/sample_submission.csv')
output.set_index('ParcelId',inplace=True)
clf.reset_parameter({"num_threads":1})
for mon in [10,11,12]:
    X_test['sale_month']=mon/12;
    p_test = clf.predict(X_test)
    output['2016'+str(mon)]=p_test
output.to_csv('lgb0605.csv', float_format='%.4f')